Importing Library for our project

In [118]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [119]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Extract features (mfcc, chroma, mel) from a sound file

In [120]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

Emotions in the RAVDESS dataset

In [121]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

Load the data and extract features for each sound file

In [122]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Data sets/Revdess Data set/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

Spliting data sets into training and testing

In [123]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

Observe the shape of the training and testing datasets:

In [124]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


And get the number of features extracted.

In [125]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


Now, let’s initialize an MLPClassifier

In [126]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

Fit/train the model.

In [127]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

Let’s predict the values for the test set. This gives us y_pred

In [128]:
y_pred=model.predict(x_test)

In [129]:
y_pred

array(['happy', 'happy', 'calm', 'calm', 'fearful', 'happy', 'happy',
       'disgust', 'disgust', 'calm', 'fearful', 'disgust', 'calm', 'calm',
       'happy', 'disgust', 'disgust', 'disgust', 'fearful', 'fearful',
       'fearful', 'disgust', 'fearful', 'fearful', 'calm', 'disgust',
       'happy', 'disgust', 'fearful', 'calm', 'disgust', 'disgust',
       'happy', 'calm', 'happy', 'disgust', 'calm', 'disgust', 'calm',
       'happy', 'happy', 'fearful', 'calm', 'disgust', 'disgust',
       'fearful', 'disgust', 'disgust', 'calm', 'disgust', 'fearful',
       'fearful', 'fearful', 'disgust', 'happy', 'fearful', 'disgust',
       'happy', 'calm', 'happy', 'fearful', 'happy', 'disgust', 'calm',
       'fearful', 'calm', 'happy', 'fearful', 'disgust', 'calm',
       'fearful', 'fearful', 'fearful', 'fearful', 'fearful', 'disgust',
       'disgust', 'happy', 'fearful', 'happy', 'disgust', 'disgust',
       'happy', 'fearful', 'disgust', 'happy', 'disgust', 'happy',
       'fearful', 'cal

To calculate the accuracy of our model, we’ll call up the accuracy_score() function we imported from sklearn. Finally, we’ll round the accuracy to 2 decimal places and print it out.

In [130]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 73.96%


In [131]:
from sklearn.metrics import accuracy_score, f1_score

In [132]:
f1_score(y_test, y_pred,average=None)

array([0.74698795, 0.68888889, 0.77876106, 0.73469388])

In [133]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,fearful,happy
2,calm,calm
3,calm,calm
4,fearful,fearful
5,happy,happy
6,happy,happy
7,happy,disgust
8,disgust,disgust
9,calm,calm


In [134]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [135]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Data sets/Revdess Data set/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [136]:
feature

array([[-7.26217224e+02,  6.85414200e+01,  3.29339767e+00,
         1.22053003e+01,  5.51027775e+00,  1.36674099e+01,
        -2.98382854e+00,  3.09802914e+00, -3.31081343e+00,
        -1.56438446e+00, -7.86165237e+00, -2.12428164e+00,
         2.84920382e+00, -2.66780639e+00,  9.59020197e-01,
         1.62816656e+00, -2.73668885e+00,  2.54240990e-01,
         2.67537737e+00, -1.76116550e+00, -1.88647425e+00,
        -9.75619972e-01, -3.79437268e-01,  4.00272876e-01,
        -3.04404378e+00, -2.90125823e+00, -1.09248057e-01,
        -8.63565266e-01, -3.33326936e+00, -1.97846520e+00,
         4.57081497e-01, -1.39910948e+00, -2.92685509e+00,
         1.39567470e-02, -4.90733683e-01, -5.70905447e-01,
         4.03992198e-02, -1.20721745e+00, -1.59498155e+00,
        -1.43648744e+00,  7.62991130e-01,  7.86087215e-01,
         7.68469393e-01,  7.68762171e-01,  7.73639500e-01,
         7.66558707e-01,  7.70558536e-01,  7.64969945e-01,
         7.80013800e-01,  7.60865748e-01,  7.59408653e-0